# Imports

In [38]:
import os
import pandas as pd
import pickle

# Data parser

In [39]:
peoples = []
files = os.listdir(os.getcwd())
for file in files:
    if(len(peoples) > 10):
        break
    if 'pums80' in file:
        current_peoples = []
        with open(os.path.join(os.getcwd(), file), 'r') as content:
            data = content.read().split('\n')
            for i in data:
                if 'P0' in i:
                    peoples.append(i)
peoples
            


['...-999500000000000000000000000-...',
 ...]

# Data preparation

In [40]:
dataset = {}
married = []
children = []
graduation = []
race = []
for people in peoples:
    marr = int(people[10])
    if(marr == 4): #не женат
        marr = 0
    elif(marr == 0): #женат
        marr = 1
    elif(marr == 1): #овдовел
        marr = 1
    elif(marr == 3): #разделены
        marr = 2
    elif(marr == 2): #в разводе
        marr = 3
    married.append(marr)

    childs = 5 if int(people[31] + people[32]) > 5 else int(people[31] + people[32])
    children.append(childs)

    grad = 8 if int(people[39] + people[40]) - 2 > 8 else int(people[39] + people[40]) - 2
    graduation.append(grad)

    rac = int(people[11] + people[12])
    if(rac > 2 & rac != 12):
        rac = 0
    race.append(rac)
    

df = pd.DataFrame({'married': married, 'childrens': children, 'graduation': graduation, 'race': race})

df.head()

,married,childrens,graduation,race
0,1,3,8,0
1,1,0,8,0
2,0,0,8,0
3,0,0,8,0
4,0,0,8,0


# Predict wage

In [41]:
#load model
with open('model.pkl', 'rb') as file:
    # Call load method to deserialze
    log_reg = pickle.load(file)
    print(log_reg)

LogisticRegression(random_state=40, solver='liblinear')


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.1.3 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [42]:
df['wage'] = log_reg.predict(df[['race', 'graduation', 'childrens', 'married']])
df.head()

,married,childrens,graduation,race,wage
0,1,3,8,0,2
1,1,0,8,0,2
2,0,0,8,0,1
3,0,0,8,0,1
4,0,0,8,0,1
